In [120]:
import os
import numpy as np
import gym
import matplotlib.pyplot as plt
%matplotlib inline


In [121]:
tsp_data_txt = open('tsp_dataset.txt')

In [122]:
lines = tsp_data_txt.readlines()

In [123]:
tsp_data_txt = open('tsp_dataset.txt')
data = {}
lines = tsp_data_txt.readlines()
distance_matrix = np.zeros((len(lines),len(lines)))
for i,line in enumerate(lines):
    for j,element in enumerate((line.split('\n')[0].split('       '))):
        if(element == ''):
            continue
        
        distance_matrix[i][j-1] = int(element.strip())
data['distance_matrix']= distance_matrix
data['reward_matrix'] = [1 for i  in range(0,len(distance_matrix))]
data['depot'] = 0
data['n_state'] = sum(list(range(1,len(distance_matrix))))
data['n_action'] = len(distance_matrix)
data['visited_nodes'] = []


In [124]:
for i in range(0,len(data['distance_matrix'])):
    data['distance_matrix'][i][i] = 9999


In [125]:
def change_reward(envp):
    for i in range(0,envp['n_state']):
        for j in range(0,envp['n_action']):
            for k in range(0,envp['n_action']):
                if k==0:
                    env['P'][i][j][k][2]= 10
                    env['P'][i][j][k][3]= True
                    break
    return envp

In [130]:
def initialze_value(env):
    initial_P = 1/(env['n_action']-1)
    env['P'] = {}
    
    #P[state][action] = [(prob, next state, reward, done), ...]
    
    for i in range(0,env['n_state']):
        env['P'][i] ={}
        
        if(i==104):
            for j in range(0,env['n_action']):
                env['P'][i][j] = []
                initial_P = 1/(env['n_action']-1)
                for k in range(0,env['n_action']):
                    if (j == env['depot'] and k == env['depot']):
                        ini_temp = [1,k,10,True]
                        env['P'][i][j].append(ini_temp)
                        initial_P = 0
                        continue
                    ini_temp = [initial_P,k,data['reward_matrix'][k],False]
                    env['P'][i][j].append(ini_temp)
        else:
            for j in range(0,env['n_action']):
                env['P'][i][j] = []
                for k in range(0,env['n_action']):
                    ini_temp = [initial_P,k,data['reward_matrix'][k],False]
                    env['P'][i][j].append(ini_temp)
            
            
    return env
    

In [131]:
env['P'][104][0]

[[1, 0, 10, True],
 [0, 1, 1, False],
 [0, 2, 1, False],
 [0, 3, 1, False],
 [0, 4, 1, False],
 [0, 5, 1, False],
 [0, 6, 1, False],
 [0, 7, 1, False],
 [0, 8, 1, False],
 [0, 9, 1, False],
 [0, 10, 1, False],
 [0, 11, 1, False],
 [0, 12, 1, False],
 [0, 13, 1, False],
 [0, 14, 1, False]]

In [132]:
env = initialze_value(data)

In [59]:
class SARSAAgent():
    def __init__(self,n_state,n_action,epsilon=1,alpha=0.1,gamma=0.99):
        self.n_state = n_state
        self.n_action= n_action
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma =gamma
        self.Q = np.zeros([n_state,n_action])
        
    def update_Q(self, state,action,reward,state_prime, action_prime,done):
        Q_old = self.Q[state,action]
        if done :
            td_target = reward
        else :
            td_target = self.gamma*self.Q[state_prime,action_prime]
        td_error = td_target - Q_old
        self.Q[state,action] = Q_old+self.alpha*td_error
        
        #choose E- greedy policy
    def update_epsilon(self, epsilon):
        self.epsilon = np.min([epsilon,1])

    def get_action(self,state):
        if np.random.uniform() < self.epsilon:
            action = np.random.randint(0,high=self.n_action)
        else:
            action = np.argmax(self.Q[state])
        return action
        

In [134]:
n_state = env['n_state']
n_action = env['n_action']
agent = SARSAAgent(n_state,n_action,epsilon=1.0,alpha=0.1,gamma=0.999)

In [218]:
n_episiode = 1000
ini_env = env
for e_idx in range(n_episiode):
    env['visited_nodes'] = []
    env = ini_env
    state = env['depot']
    env['visited_nodes'].append(state)
    for s in range(0,env['n_state']-1):
        for vn in env['visited_nodes']:
            env['P'][s][vn][vn] = [0,vn,-1,False]
    action = agent.get_action(state)
    done = False
    while not done:
        _,state_prime,reward,done =env['P'][state][action][action]
        
        if state_prime+state > env['n_state']-1:
            action = agent.get_action(state)
            continue
                                      
        state_prime = state_prime+state
        action_prime = agent.get_action(state_prime)
        agent.update_Q(state,action,reward,state_prime,action_prime,done)
        env['visited_nodes'].append(state_prime-state)
        state = state_prime
        action = action_prime
        if(state==104):
            done= True
    agent.update_epsilon(100/(e_idx+1))
print("sarsa done.")

sarsa done.


In [226]:
# Test
gamma = 0.99
env = ini_env
ret = 0
state = env['depot']
visited_list = []
for tick in range(1000):
    print("\n tick:[{}]".format(tick))
    print(visited_list)
    action = agent.get_action(state) # select action
    if action not in  visited_list:
        visited_list.append(action)
    _,next_obs,reward,done = env['P'][state][action][action]
    obs = next_obs+state
    ret = reward + gamma*ret 
    state = next_obs
    
    if len(visited_list)== 15: break
print ("Return is [{:.3f}]".format(ret))


 tick:[0]
[]

 tick:[1]
[0]

 tick:[2]
[0]

 tick:[3]
[0]

 tick:[4]
[0]

 tick:[5]
[0]

 tick:[6]
[0]

 tick:[7]
[0, 7]

 tick:[8]
[0, 7]

 tick:[9]
[0, 7]

 tick:[10]
[0, 7]

 tick:[11]
[0, 7]

 tick:[12]
[0, 7]

 tick:[13]
[0, 7]

 tick:[14]
[0, 7]

 tick:[15]
[0, 7]

 tick:[16]
[0, 7]

 tick:[17]
[0, 7]

 tick:[18]
[0, 7]

 tick:[19]
[0, 7]

 tick:[20]
[0, 7]

 tick:[21]
[0, 7]

 tick:[22]
[0, 7]

 tick:[23]
[0, 7]

 tick:[24]
[0, 7]

 tick:[25]
[0, 7]

 tick:[26]
[0, 7]

 tick:[27]
[0, 7]

 tick:[28]
[0, 7]

 tick:[29]
[0, 7]

 tick:[30]
[0, 7]

 tick:[31]
[0, 7]

 tick:[32]
[0, 7]

 tick:[33]
[0, 7]

 tick:[34]
[0, 7]

 tick:[35]
[0, 7]

 tick:[36]
[0, 7]

 tick:[37]
[0, 7]

 tick:[38]
[0, 7]

 tick:[39]
[0, 7]

 tick:[40]
[0, 7]

 tick:[41]
[0, 7]

 tick:[42]
[0, 7]

 tick:[43]
[0, 7, 4]

 tick:[44]
[0, 7, 4]

 tick:[45]
[0, 7, 4]

 tick:[46]
[0, 7, 4]

 tick:[47]
[0, 7, 4]

 tick:[48]
[0, 7, 4, 9]

 tick:[49]
[0, 7, 4, 9]

 tick:[50]
[0, 7, 4, 9, 10]

 tick:[51]
[0, 7, 4, 9, 1

In [225]:
visited_list

[0, 12, 1, 10, 13, 7, 9, 6, 8, 11, 3, 14, 2, 4, 5]